<a href="https://colab.research.google.com/github/JocznHM/Inteligencia-artificial-CJHM/blob/main/final_IA_CJHM19180909.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Codigo fuente del proyecto final de IA

In [ ]:
"""
Author: JocznHM
Version: 1.0

"""
#IMPORTS
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflowjs as tfjs


# Función para cargar y preprocesar imágenes
def load_images(dataset_path):
    images = [] #arreglo de imagenes
    labels = [] #arreglo de etiquetas del dataset
    for label, folder in enumerate(["normal", "sospechoso"]):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Advertencia: No se pudo cargar la imagen {img_path}.")
                continue
            img = cv2.resize(img, (224, 224))  # Redimensionar a 224x224
            img = img / 255.0  # Normalizar la imagen
            images.append(img)
            labels.append(label)  # 0: normal, 1: sospechoso

    #se retorna una tupla de dos arreglos empleando los arreglos de numpy
    return np.array(images), np.array(labels)

#Función para crear el modelo
def create_model():
    #se establece el modelo como secuencial
    model = Sequential()
    #se agregan distintas capas dentro de la red neuronal y se ajustan la densidad de algunas capas
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))  # 2 clases: normal, sospechoso
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    #se retorna un modelo
    return model

# Predecir una imagen
def predict_image(image_path, model):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)  # Expandir dimensiones para el modelo
    prediction = model.predict(img)
    return "Normal" if np.argmax(prediction) == 0 else "Sospechoso"


# Exportar el modelo a TensorFlow.js
def export_model_to_tfjs(model, export_dir="tfjs_model"):
    """
    Exporta el modelo entrenado al formato compatible con TensorFlow.js.

    Args:
        model: Modelo de TensorFlow entrenado.
        export_dir: Directorio donde se guardará el modelo exportado.
    """
    try:
        tfjs.converters.save_keras_model(model, export_dir)
        print(f"Modelo exportado exitosamente a {export_dir}")
    except Exception as e:
        print(f"Error al exportar el modelo: {e}")

# Entrenamiento del modelo
if __name__ == "__main__":
    dataset_path = "./dataset/"  # Ruta de mi dataset o imagenes de entrenamiento
    print("Cargando imágenes...")
    images, labels = load_images(dataset_path)
    labels = to_categorical(labels, num_classes=2)

    print("Creando el modelo...")
    model = create_model()

    #Este es un metodo que se aplica cuando se tienen muy pocas imagenes
    #para el entrenamiento del modelo
    print("Aplicando data augmentation...")
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        validation_split=0.1  # Usar 10% de datos para validación
    )

    #Entrenamiento del modelo
    train_gen = datagen.flow(images, labels, batch_size=32, subset='training')
    val_gen = datagen.flow(images, labels, batch_size=32, subset='validation')

    print("Entrenando el modelo...")
    model.fit(train_gen, validation_data=val_gen, epochs=20)
    #Se guarda el modelo para ser usado dentro de una apgina web con tensorflowjs
    export_model_to_tfjs(model, export_dir="tfjs_model")

    # Opciones de predicción
    # Simplemente es codigo de prueba para verificar que funcione antes de subir el programa
    while True:
        print("\nOpciones:")
        print("1. Predecir una imagen desde archivo")
        print("2. Salir")
        choice = input("Selecciona una opción (1/2): ")

        if choice == "1":
            image_path = input("Ingresa la ruta de la imagen: ")
            result = predict_image(image_path, model)
            print(f'La imagen es: {result}')
        elif choice == "2":
            print("Saliendo del programa...")
            break
        else:
            print("Opción no válida. Intenta nuevamente.")


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Clasificador de comportamiento</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            text-align: center;
            margin: 20px;
        }
        .container {
            max-width: 600px;
            margin: 0 auto;
            padding: 20px;
            border: 1px solid #ddd;
            border-radius: 10px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        }
        input[type="file"] {
            display: none;
        }
        .custom-file-upload {
            padding: 12px 25px;
            font-size: 16px;
            background-color: #28a745;
            color: white;
            border: none;
            border-radius: 8px;
            cursor: pointer;
            transition: background-color 0.3s, transform 0.3s;
            margin-top: 20px;
        }
        .custom-file-upload:hover {
            background-color: #218838;
            transform: scale(1.05);
        }
        .custom-file-upload:focus {
            outline: none;
            box-shadow: 0 0 0 3px rgba(40, 167, 69, 0.5);
        }
        .custom-file-upload:active {
            background-color: #1e7e34;
        }
        img {
            max-width: 100%;
            height: auto;
            border: 1px solid #ddd;
            border-radius: 10px;
            margin-bottom: 20px;
        }
        .result {
            font-size: 1.2em;
            margin-top: 20px;
        }
        /* Estilo del botón de limpiar */
        button {
            padding: 12px 25px;
            font-size: 16px;
            background-color: #007BFF;
            color: white;
            border: none;
            border-radius: 8px;
            cursor: pointer;
            transition: background-color 0.3s, transform 0.3s;
            margin-top: 20px;
        }
        button:hover {
            background-color: #0056b3;
            transform: scale(1.05);
        }
        button:focus {
            outline: none;
            box-shadow: 0 0 0 3px rgba(38, 143, 255, 0.5);
        }
        button:active {
            background-color: #004085;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Clasificador de comportamiento</h1>
        <p>Sube una imagen para determinar si el comportamiento es <strong>sospechoso</strong> o <strong>normal</strong>.</p>

        <!-- Botón personalizado para cargar la imagen -->
        <label for="imageInput" class="custom-file-upload">
            Cargar Imagen
        </label>
        <input type="file" id="imageInput" accept="image/*">

        <div>
            <img id="previewImage" alt="Image Preview" style="display: none;">
        </div>

        <div class="result" id="result">El resultado aparecerá aquí.</div>

        <!-- Botón de limpiar con estilo mejorado -->
        <button id="clearButton">Limpiar</button>
    </div>

    <script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs"></script>
    <script>
        let model;

        // Carga el modelo TensorFlow.js
        async function loadModel() {
            const modelPath = './tfjs_model/model.json'; // Ajusta la ruta al modelo exportado
            model = await tf.loadLayersModel(modelPath);
            console.log('Model loaded successfully');
        }

        // Evento que detecta cuando la imagen se sube
        document.getElementById('imageInput').addEventListener('change', async (event) => {
            const file = event.target.files[0];
            if (file) {
                // Vista previa de la imagen
                const previewImage = document.getElementById('previewImage');
                previewImage.src = URL.createObjectURL(file);
                previewImage.style.display = 'block';

                // Pasa la imagen al tensor
                const image = await readImageAsTensor(file);

                // Hace la predicción
                const prediction = model.predict(image);
                const probabilities = prediction.arraySync()[0];

                // Muestra el resultado
                const resultDiv = document.getElementById('result');
                if (probabilities[0] > probabilities[1]) {
                    resultDiv.textContent = 'Resultado: comportamiento normal detectado.';
                } else {
                    resultDiv.textContent = 'Resultado: Comportamiento sospechoso detectado!';
                }

                // Limpiar
                image.dispose();
            }
        });

        // Función para leer la imagen como tensor
        async function readImageAsTensor(file) {
            return new Promise((resolve) => {
                const reader = new FileReader();
                reader.onload = () => {
                    const img = new Image();
                    img.onload = () => {
                        const tensor = tf.browser.fromPixels(img)
                            .resizeNearestNeighbor([224, 224]) // Redimensiona a las dimensiones del modelo
                            .toFloat()
                            .div(tf.scalar(255.0)) // Normaliza
                            .expandDims(); // Añade la dimensión del lote
                        resolve(tensor);
                    };
                    img.src = reader.result;
                };
                reader.readAsDataURL(file);
            });
        }

        // Inicializar la aplicación cargando el modelo
        loadModel();

        // Evento para limpiar la imagen y el resultado
        document.getElementById('clearButton').addEventListener('click', () => {
            // Limpiar la imagen de vista previa
            const previewImage = document.getElementById('previewImage');
            previewImage.style.display = 'none';
            previewImage.src = ''; // Limpiar la fuente de la imagen

            // Limpiar el campo de entrada de archivos
            const imageInput = document.getElementById('imageInput');
            imageInput.value = ''; // Resetea el input de archivo

            // Limpiar el resultado
            const resultDiv = document.getElementById('result');
            resultDiv.textContent = 'El resultado aparecerá aquí.';
        });
    </script>
</body>
</html>


In [ ]:
tensorflow==2.8
keras==2.8
numpy==1.21.6
pillow==8.4.0
matplotlib==3.5.1
h5py==3.1.0
scikit-learn==1.0.2
opencv-python==4.5.5.64
tensorflow-datasets==4.5.2
tensorflow-hub==0.12.0
tensorflowjs==3.18.0